# Assignment 1. Computation of journal bearing

It is possible to use the Reynolds equation to compute de eccentricity and the angle of a journal bearing with fluid film lubrication, given the load $W$

![image.png](Journal_bearing1.png)

Consider that the journal is displaced in the $x$ negative direction, so that $h(\theta) = h_0(1+\varepsilon \cos(\theta))$ where $\varepsilon=\frac{e}{h_0}$ is the relative eccentricity and $h_0 = \frac{1}{2}(D_b -D_a)$ is the average gap thickness. The journal is rotating in the positive direction of $\theta$ (polar coordinates)

![image.png](Journal_bearing2.png)

1. Calculate the pressure distribution $p(\theta)$ and the value and direction of $\mathbf{W}$, in the approximation $\varepsilon \ll 1$.
2. What happens to the journal if $\mathbf{W}$ increase a little bit?
3. Apply the results to this real [example](https://www.mcmaster.com/6658K738/), and discuss the value of the maximum load given by the manufactures at this rotational velocity.
![image.png](McMaster-Carr_JB.png)

## Solution

### 1. Pressure distribution and value and direction of W
#### 1.1. Pressure distribution

From the theory notes, the differential equation of the pressure distribution is already known:

In [1]:
try:
    import sympy as sp
except:
    !pip install sympy
    import sympy as sp

In [2]:
x,mu,U,q = sp.symbols('x,mu,U,q')
p = sp.Function('p')
u = sp.Function('u')
h = sp.Function('h')
ode = sp.Eq(p(x).diff(x),(6*mu*(U*h(x)-2*q))/(h(x)**3))
display(ode)

Eq(Derivative(p(x), x), 6*mu*(U*h(x) - 2*q)/h(x)**3)

For this journal bearing problem, as $p(\theta)$ is the desired function, the following change in the coordinate system can be made:
$$
x = R_a\theta = \frac{D_a}{2}\theta\\
dx = \frac{D_a}{2}d\theta\\
U = \Omega R_a = \Omega \frac{D_a}{2}
$$
Making this change in the previous equation, it is obtained:

In [3]:
Da,theta,Omega = sp.symbols('D_a,theta,Omega')
ode_theta = sp.Eq(1/(Da/2)*p(theta).diff(theta),(6*mu*((Omega*Da/2)*h(theta)-2*q))/(h(theta)**3))
display(ode_theta)

Eq(2*Derivative(p(theta), theta)/D_a, 6*mu*(D_a*Omega*h(theta)/2 - 2*q)/h(theta)**3)

In [4]:
ode_theta = sp.Eq(p(theta).diff(theta),Da/2*(6*mu*((Omega*Da/2)*h(theta)-2*q))/(h(theta)**3))
display(ode_theta)

Eq(Derivative(p(theta), theta), 3*D_a*mu*(D_a*Omega*h(theta)/2 - 2*q)/h(theta)**3)

Then, the geometry can be introduced with $h(\theta) = h_0(1+\epsilon\cos{\theta})$ to the previous equation:

In [5]:
h0,eps = sp.symbols('h_0,epsilon')
ode_theta = ode_theta.replace(h(theta),h0*(1+eps*sp.cos(theta)))
display(ode_theta)

Eq(Derivative(p(theta), theta), 3*D_a*mu*(D_a*Omega*h_0*(epsilon*cos(theta) + 1)/2 - 2*q)/(h_0**3*(epsilon*cos(theta) + 1)**3))

Now, the approximation $\epsilon\ll1$ is applied by means of Taylor expansion series of $\epsilon$ around 0 up to the linear factor:

In [6]:
ode_approx_lhs = ode_theta.lhs
ode_approx_rhs = ode_theta.rhs.series(eps,0,n=2).removeO()
ode_approx = sp.Eq(ode_approx_lhs,ode_approx_rhs)
display(ode_approx)

Eq(Derivative(p(theta), theta), 3*D_a**2*Omega*mu/(2*h_0**2) - 6*D_a*mu*q/h_0**3 + epsilon*(-3*D_a**2*Omega*mu*cos(theta)/h_0**2 + 18*D_a*mu*q*cos(theta)/h_0**3))

Now, the differential equation can be solved for $p(\theta)$:

In [8]:
ptheta_ind = sp.dsolve(ode_approx,p(theta))
display(ptheta_ind)

Eq(p(theta), C1 + 3*D_a*mu*(2*epsilon*(-D_a*Omega*h_0 + 6*q)*sin(theta) + theta*(D_a*Omega*h_0 - 4*q))/(2*h_0**3))

This is the indefinite integral. Then, at $\theta = 0º$ some unknown pressure $p_0$ may be found, hence:

In [14]:
p0 = sp.symbols('p_0')
ptheta = sp.dsolve(ode_approx,p(theta),ics={p(0):p0})
display(ptheta)

Eq(p(theta), 3*D_a*mu*(2*epsilon*(-D_a*Omega*h_0 + 6*q)*sin(theta) + theta*(D_a*Omega*h_0 - 4*q))/(2*h_0**3) + p_0)

After one revolution, i.e. $\theta = 2\pi$, it is seen that the pressure will also be $p_0$: $p(\theta = 2\pi) = p_0$. From this boundary condition and the previous equation, the flow rate $q$ can be found:

In [17]:
qexp = sp.solve(ptheta.replace(theta,2*sp.pi).replace(p(2*sp.pi),p0),q)
display(qexp[0])

D_a*Omega*h_0/4

This flow rate $q$ is replaced in the previous equation of $p(\theta)$:

In [18]:
ptheta = ptheta.replace(q,qexp[0])
display(ptheta)

Eq(p(theta), 3*D_a**2*Omega*epsilon*mu*sin(theta)/(2*h_0**2) + p_0)

#### 1.2. Value and direction of W

In order to find the forces generated by the pressure distribution, these can be computed by its integration on the surface that receives the force. Supposing unit length in the z-axis, then:

$$
F_x = \int_{0}^{2\pi} p_x(\theta)\:R_a\:d\theta = \int_{0}^{2\pi} p_x(\theta)\:\frac{D_a}{2}\:d\theta\\
F_y = \int_{0}^{2\pi} p_y(\theta)\:R_a\:d\theta = \int_{0}^{2\pi} p_y(\theta)\:\frac{D_a}{2}\:d\theta
$$

As the pressure acts directly in the orthogonal direction of a differential element of surface of the journal, then (according to the axes of the second figure):

$$
p_x(\theta) = -p(\theta)\cos{\theta}\\
p_y(\theta) = -p(\theta)\sin{\theta}
$$

The forces are:

In [25]:
px = -ptheta.rhs*sp.cos(theta)
py = -ptheta.rhs*sp.sin(theta)

In [26]:
Fx = px.integrate((theta,0,2*sp.pi))*Da/2
display(Fx)

0

The journal does not receive any force in the x direction.

In [27]:
Fy = py.integrate((theta,0,2*sp.pi))*Da/2
display(Fy)

-3*pi*D_a**3*Omega*epsilon*mu/(4*h_0**2)

The journal does receive a force in the negative direction of the y-axis (force per unit length in the z direction).